In [1]:
import warnings; warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
import nltk
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [3]:
from pathlib import Path
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pandas as pd

In [5]:
path = Path('C:/Users/sappusamy/Documents/SriWK/datasets/IMDB.csv')

In [6]:
dataset = pd.read_csv(path)

In [7]:
dataset.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [8]:
dataset = dataset[:250]

In [9]:
dataset.sentiment.value_counts()

negative    137
positive    113
Name: sentiment, dtype: int64

In [10]:
train = dataset[:100]
train.sentiment.value_counts()

negative    58
positive    42
Name: sentiment, dtype: int64

In [11]:
test = dataset[100:].reset_index(drop=True)
test.sentiment.value_counts()

negative    79
positive    71
Name: sentiment, dtype: int64

## NLP: Cleaning & Preprocessing text
- Remove HTML
- Tokenization + Remove punctuation
- Remove stopwords
- Lemmatization or stemming

In [12]:
text = train.review[0]

In [13]:
stop_words = stopwords.words('english')

In [14]:
def remove_html(text):
    return BeautifulSoup(text,'lxml').get_text()

In [15]:
def remove_punctuation(text):
    return "".join([c for c in text if c not in string.punctuation])

In [16]:
def remove_stopwords(text):
    return [w for w in text if w not in stop_words]

In [17]:
def word_lemmatizer(text):
    return [lemmatizer.lemmatize(i) for i in text]

In [18]:
def word_stemmer(text):
    return [stemmer.stem(i) for i in text]

In [19]:
tokenizer = RegexpTokenizer(r'\w+')

In [20]:
lemmatizer = WordNetLemmatizer()

In [21]:
stemmer = PorterStemmer()

In [22]:
t = train['review'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [23]:
labels = train['sentiment']

set(labels)

{'negative', 'positive'}

In [24]:
labels_index = {'negative':0,'positive':1}

#### Represent each word in ONE-HOT encoding
- as of now, for reducing vector length of input we set min frequency of token to be **SOME INT VALUE**

In [25]:
def token_frequency(l):
    count = {}
    for tokens in l:
        for token in tokens:
            if token not in count:
                count[token]=1
            else:
                count[token]+=1
    return count

In [26]:
tf = token_frequency(t)

In [27]:
sorted(tf.items(),key=lambda x:x[1],reverse=True)[:10]

[('movi', 209),
 ('film', 156),
 ('one', 104),
 ('like', 84),
 ('see', 59),
 ('even', 59),
 ('get', 58),
 ('good', 56),
 ('scene', 55),
 ('go', 54)]

In [28]:
min_freq = 2

In [29]:
def build_vocabulary(tf):
    v = {}
    v['pad']=0
    v['unk']=1
    for token in tf:
        if tf[token]>=min_freq:
            v[token] = len(v)
    return v

In [30]:
vocab = build_vocabulary(tf)

In [31]:
len(vocab)

1665

## Let's convert tokens to index and make vector for each word in a review

In [32]:
tokens = t[0]
tokens[:10],len(tokens)

(['one',
  'review',
  'mention',
  'watch',
  '1',
  'oz',
  'episod',
  'youll',
  'hook',
  'right'],
 168)

In [33]:
indices = [vocab[token] if token in vocab else vocab['unk'] for token in tokens]
len(indices)

168

## Let's convert these indices to one-hot vectors
- shape of input will be sequence_length * vocab_size

In [34]:
x = torch.zeros(len(indices),len(vocab))

In [35]:
x[range(len(indices)),indices]=1

In [36]:
x,x.shape

(tensor([[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]), torch.Size([168, 1665]))

## Equation of RNN:
\begin{equation*}
h_t = tanh( W_{ih}X_t + b_{ih} + W_{hh}h_{t-1} + b_{hh})
\end{equation*}

In [37]:
rnn_cell = nn.RNNCell(len(vocab),100)

In [38]:
for name,p in rnn_cell.named_parameters():
    print(name,p.shape)

weight_ih torch.Size([100, 1665])
weight_hh torch.Size([100, 100])
bias_ih torch.Size([100])
bias_hh torch.Size([100])


In [39]:
h = torch.zeros(1,100)

In [40]:
h

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [41]:
for i in range(len(indices)):
    h = rnn_cell(x[i].view(1,-1),h)

In [42]:
print("**value of hidden vector after iterating over all input time steps**")
h

**value of hidden vector after iterating over all input time steps**


tensor([[ 0.0131, -0.0322, -0.1322, -0.0975,  0.0204,  0.1817,  0.0846,  0.0201,
         -0.0918,  0.0665,  0.0676, -0.0776, -0.0997,  0.0302, -0.1368,  0.2011,
          0.1128, -0.0234,  0.1890,  0.0898,  0.0630, -0.0489, -0.1366,  0.1285,
         -0.0894,  0.0062,  0.0213,  0.0666,  0.1108,  0.0107, -0.0342,  0.0358,
         -0.1576,  0.1320, -0.0721,  0.2109, -0.0256,  0.0909, -0.0953,  0.2060,
         -0.0171,  0.0429, -0.1006, -0.0366, -0.0918,  0.2314,  0.0716,  0.0137,
         -0.0338,  0.0475,  0.0308,  0.0188, -0.0686,  0.0243,  0.0420,  0.0932,
         -0.0346, -0.0535,  0.1153, -0.1892, -0.0197,  0.1771,  0.0563,  0.1012,
          0.1362, -0.2485, -0.1255,  0.0852,  0.0730,  0.0133,  0.2124, -0.1108,
          0.0870, -0.2446,  0.1138,  0.1723, -0.0421,  0.0054, -0.0879, -0.0137,
         -0.0378,  0.1887, -0.2409,  0.1479,  0.1549,  0.0046, -0.0304, -0.0861,
         -0.1304,  0.0440, -0.0577,  0.1125,  0.1231, -0.0426, -0.1980,  0.0276,
         -0.1516, -0.2592,  

In RNNCell,
    
    inputs are looped over each time steps

<a href="https://pytorch.org/docs/stable/nn.html?highlight=rnncell#torch.nn.RNNCell">PyTorch RNNCell reference</a>

## RNN for 1 training example

In [43]:
rnn = nn.RNN(len(vocab),100,batch_first=True)

In [44]:
indices = [vocab[token] if token in vocab else vocab['unk'] for token in tokens]

In [45]:
x = torch.zeros(len(indices),len(vocab))

x[range(len(indices)),indices]=1


In [46]:
x = x.expand((1,-1,len(vocab)))
x,x.shape

(tensor([[[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]), torch.Size([1, 168, 1665]))

**Shape of h:-**
    
    h = (num_layers*num_directions, bacth_size, hidden_units)
    
    where num_layers parameter take int value as inputs which build ups STACKED RNN

In [47]:
h = torch.zeros(1,1,100)

In [48]:
output,hidden = rnn(x,h)

In [49]:
output.shape,hidden.shape

(torch.Size([1, 168, 100]), torch.Size([1, 1, 100]))

**output** - output of all time steps of all batches

**hidden** - output of last time step of all batches

In [50]:
classifier = nn.Linear(100,2)

In [51]:
preds = classifier(hidden.view(1,100))
preds

tensor([[-0.1841,  0.1122]], grad_fn=<AddmmBackward>)

## RNN for m training examples

In [52]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [53]:
crit = nn.CrossEntropyLoss()

In [54]:
optimizer = optim.SGD([{'params':rnn.parameters()},{'params':classifier.parameters()}],lr=0.01)

In [55]:
for doc,y in zip(indices,labels):
    x = torch.zeros(len(doc),len(vocab))
    x[range(len(doc)),doc]=1
    x = x.expand((1,-1,len(vocab)))
    
    optimizer.zero_grad()
    
    h = torch.zeros(1,1,100)
    output,hidden = rnn(x,h)
    preds = classifier(hidden.view(1,100))
    
    loss = crit(preds,torch.LongTensor([labels_index[y]]))
    loss.backward()
    
    optimizer.step()
    
    print(loss.item())
    

0.5559003353118896
0.5865741968154907
0.5760800242424011
0.881472110748291
0.5523629188537598
0.5476473569869995
0.5382099747657776
0.8598165512084961
0.8195828199386597
0.5355939865112305
0.8742824792861938
0.8003334403038025
0.8464121222496033
0.8539475202560425
0.6447441577911377
0.7875109910964966
0.6243841648101807
0.7863596081733704
0.5838578343391418
0.741943359375
0.6739190220832825
0.7623177766799927
0.6147756576538086
0.7852587699890137
0.7390691637992859
0.6596775650978088
0.6310624480247498
0.7847815752029419
0.7708404064178467
0.6257534623146057
0.604915201663971
0.6955050230026245
0.7397541999816895
0.5824265480041504
0.7451844811439514
0.7188671827316284
0.7497631311416626
0.7459926605224609
0.6593024730682373
0.7782583832740784
0.7049355506896973
0.6471307873725891
0.6932894587516785
0.6283427476882935
0.7973484992980957
0.648339569568634
0.7162867784500122
0.7163512706756592
0.7036973237991333
0.6895565986633301
0.6535287499427795
0.7471995949745178
0.6481081247329712


**Why loop over each document?**

    different documents are of differnt lengths

## Batch-wise processing

**Why do we need padding?**
- sequence length of different documents will be of different length
- so flow will be as follows:<br>
      for doc in documents:
          for token in doc:
              pass to RNNCell
- the above methods has huge complexity
- in order to process in batch-wise, we make all documents length to be of same length

    
    Thus padding comes in, which means pads 0's to documents which has length less than max length

In [56]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [57]:
sequence_lengths = [len(d) for d in indices]

In [58]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

372

In [59]:
X = torch.zeros(len(indices),max_length,len(vocab))

In [60]:
X.shape

torch.Size([100, 372, 1665])

In [61]:
for i in range(len(indices)):
    X[[i],range(len(indices[i])),indices[i]]=1

In [62]:
Y = torch.LongTensor([labels_index[i] for i in labels])

In [63]:
class Sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(len(vocab),100,batch_first=True)
        self.classifier = nn.Linear(100,2)
        
    def forward(self,x,h):
        output,hidden = self.rnn(x,h)
        return self.classifier(hidden.view(-1,100))

In [64]:
model = Sentiment()

In [65]:
crit = nn.CrossEntropyLoss()

In [66]:
optimizer = optim.SGD(model.parameters(),lr=0.1)

In [67]:
bs=16

In [68]:
for epoch in range(2):
    for i in range(0,X.shape[0],bs):
        xb = X[i:i+bs]
        yb = Y[i:i+bs]
        h = torch.zeros(1,xb.shape[0],100)
        
        optimizer.zero_grad()
        preds = model(xb,h)
        loss = crit(preds,yb)
        loss.backward()
        optimizer.step()
        print(loss.item())
        

0.695826530456543
0.7030476331710815
0.6810666918754578
0.6966425776481628
0.6778326034545898
0.6617869734764099
0.6349002718925476
0.7104899287223816
0.7257965803146362
0.6569234132766724
0.7050983309745789
0.6687477827072144
0.6482828855514526
0.6187847852706909


## Test

In [69]:
t = test['review'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [70]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [71]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

450

In [72]:
TY = torch.LongTensor([labels_index[i] for i in test.sentiment])

In [73]:
predictions=[]
for index_num in range(len(test)):

    T = torch.zeros(len(indices[index_num]),len(vocab))
    T[range(len(indices[index_num])),indices[index_num]]=1
    T = T.expand((1,-1,len(vocab)))

    h = torch.zeros(1,T.shape[0],100)
    with torch.no_grad():
        predictions.append(F.softmax(model(T,h)).argmax().item())

In [74]:
predictions = torch.LongTensor(predictions)

In [75]:
predictions

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])

In [76]:
"Accuracy:",(predictions==TY).float().mean()

('Accuracy:', tensor(0.5267))

In [77]:
"Number of positives predicted",(predictions==1).sum()

('Number of positives predicted', tensor(0))

In [78]:
"Number of negatives predicted",(predictions==0).sum()

('Number of negatives predicted', tensor(150))

## Use nn.Embedding

In [79]:
t = train['review'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [80]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [81]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

372

In [82]:
for i in range(len(indices)):
    if len(indices[i])<max_length:
        indices[i]+=[0]*(max_length-len(indices[i]))

In [83]:
X = torch.LongTensor(indices)
X.shape

torch.Size([100, 372])

In [84]:
Y = torch.LongTensor([labels_index[i] for i in labels])

In [85]:
class Sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(len(vocab),200,padding_idx=0)
        self.gru = nn.GRU(200,100,batch_first=True)
        self.classifier = nn.Linear(100,2)
        
    def forward(self,x,h):
        x = self.emb(x)
        output,hidden = self.gru(x,h)
        return self.classifier(hidden.view(-1,100))

In [86]:
model = Sentiment()

crit = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr=0.1)

bs=32

for epoch in range(10):
    for i in range(0,X.shape[0],bs):
        xb = X[i:i+bs]
        yb = Y[i:i+bs]
        h = torch.zeros(1,xb.shape[0],100)
        
        optimizer.zero_grad()
        preds = model(xb,h)
        loss = crit(preds,yb)
        loss.backward()
        optimizer.step()
        print(loss.item())
        

0.6929085850715637
0.7052596807479858
0.6900385618209839
0.673358142375946
0.7026670575141907
0.690097987651825
0.6701661944389343
0.6445623636245728
0.7138630747795105
0.6829387545585632
0.660126805305481
0.6280755400657654
0.7232805490493774
0.6790019273757935
0.6548410058021545
0.6183191537857056
0.7303240299224854
0.6763519048690796
0.6519342064857483
0.6123837232589722
0.7352997064590454
0.6742353439331055
0.6502665877342224
0.6086880564689636
0.738713264465332
0.672376811504364
0.64927077293396
0.6063395738601685
0.7410249710083008
0.6706810593605042
0.6486548185348511
0.60481858253479
0.7425869703292847
0.6691190004348755
0.6482614278793335
0.603813886642456
0.7436491250991821
0.6676802039146423
0.6480022072792053
0.6031358242034912


## Test

In [87]:
t = test['review'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [88]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

max_length = len(max(t,key=lambda x:len(x)))
max_length

450

In [89]:
TY = torch.LongTensor([labels_index[i] for i in test.sentiment])

In [90]:
predictions = []
for index_num in range(len(test)):
    T = torch.LongTensor([indices[index_num]])
    h = torch.zeros(1,T.shape[0],100)
    with torch.no_grad():
        predictions.append(F.softmax(model(T,h)).argmax().item())

In [91]:
predictions = torch.LongTensor(predictions)

In [92]:
(predictions==TY).float().mean()

tensor(0.5000)

In [100]:
tp = ((TY==1)*(predictions==1)).sum().item()

tn = ((TY==0)*(predictions==0)).sum().item()

fn = ((TY==1)*(predictions==0)).sum().item()

fp = ((TY==0)*(predictions==1)).sum().item()
tp,tn,fp,fn

(1, 74, 5, 70)

#### Postive label - measures

In [94]:
print("Precision:",tp/(tp+fp))

print("Recall:",tp/(tp+fn))

Precision: 0.16666666666666666
Recall: 0.014084507042253521


#### Negative label - measures

In [95]:
print("Precision:",tn/(tn+fn))

print("Recall:",tn/(tn+fp))

Precision: 0.5138888888888888
Recall: 0.9367088607594937


In [98]:
model.gru.weight_hh_l0.grad

tensor([[-2.2776e-05, -1.0590e-05,  1.2248e-05,  ...,  9.4169e-06,
         -1.9101e-05,  1.6297e-05],
        [-3.3105e-05, -1.5392e-05,  1.7803e-05,  ...,  1.3688e-05,
         -2.7764e-05,  2.3687e-05],
        [ 5.3344e-07,  2.4802e-07, -2.8687e-07,  ..., -2.2056e-07,
          4.4737e-07, -3.8169e-07],
        ...,
        [ 1.0818e-03,  5.0300e-04, -5.8179e-04,  ..., -4.4730e-04,
          9.0729e-04, -7.7408e-04],
        [-2.6151e-04, -1.2159e-04,  1.4063e-04,  ...,  1.0812e-04,
         -2.1932e-04,  1.8711e-04],
        [-1.0494e-04, -4.8793e-05,  5.6437e-05,  ...,  4.3390e-05,
         -8.8013e-05,  7.5090e-05]])

In [99]:
model.gru.weight_ih_l0.grad

tensor([[-1.4217e-25,  4.3960e-24,  2.0416e-25,  ...,  5.4143e-24,
         -1.1671e-25, -3.4600e-24],
        [ 1.5861e-24, -2.8096e-24, -7.8425e-25,  ..., -4.6890e-24,
          1.7778e-24,  3.4771e-24],
        [-9.9762e-26, -3.0760e-24,  6.0718e-25,  ..., -3.4793e-24,
         -1.3614e-24,  1.0589e-24],
        ...,
        [-4.4773e-23,  3.1713e-22,  4.0156e-23,  ...,  3.4103e-22,
          2.3316e-22, -2.7291e-22],
        [ 8.6815e-23, -1.1720e-21, -2.1915e-22,  ..., -1.2557e-21,
         -4.2305e-22,  9.4109e-22],
        [ 5.0904e-25, -9.0626e-23, -1.0189e-23,  ..., -9.3201e-23,
         -2.4840e-23,  5.1593e-23]])